In [1]:
import os
import glob
import json
import numpy as np
import copy
import re

In [179]:
domains = 'com|co.in|com.hk|de||co.uk|co.jp|fr|com.br|it|ru|es|ca|com.mx|co.id|com.tr|com.au|pl|com.sa|nl|com.ar|com.eg|co.th|com.pk|co.za|com.my|be|gr|com.vn|co.ve|com.tw|com.ua|at|se|com.co|ro|ch|pt|com.ph|cl|com.ng|com.sg|com.pe|ae|co.kr|co.hu|ie|dk|no|co.il|fi|cz|co.ma|sk|co.nz|com.kw|lk|bg|com.qa|az|kz|com.do|hr|by|com.ec|lt|iq|co.ke|com.bd|com.om|tn|si|co.cr|com.gt|com.pr|com.sv|lv|com.uy|jo|com.bo|ba|com.cu|rs|com.ly|cm|ee|co.ug|com.bh|com.np|com.gh|dz|lu|com.lb|co.uz|ci|com.py|com.ni|hn|com.et|tt|co.tz|mg|sn|cd|com.kh|am|com.af|ge|mu|com.bn|co.mz|com.jm|com.gi|is|com.pa|md|ps|com.na|mn|com.mt|co.bw|bj|kg|ml|rw|co.zm|bs|ht|la|com.bz|co.zw|as|cat|mk|ne|mw|tg|co.ao|gp|gy|bf|ga|li|co|tm|dj|mv|hk|sc|dm|bi|co.vi|vu|ad|com.vc|com.ag|com.fj|to|cf|fm|tk|gg|ws|vg|im|nu|gm|je|ms|me|co.im|tl|com.ai|gl|co.ls|co.je|st|it.ao|com.by|com.tj|pn|sh|com.sl|nr|sm|cg|co.ck|com.sb|com.cy|so|com.nf|com.ve|com.iq|jp|ac|com.tn|in|td'

pattern = u'^http[s]*://(www.|www.clck|ru.|go.|nova.|)(?P<ref>[\w]+).(%s)(?P<tail>[\w/=%%-]*)'%domains

r = re.compile(pattern)

In [8]:
def get_relation_mat(data_t):
    N=len(data_t)
    r  = re.compile('^http[s]*://[ru.]*bookimed.com(?P<path>/[\w/=-]*)')
    mat = np.zeros((N,N))
    for i in range(0,N):
        for j in range(i+1,N):
            try:
                ref = re.match(r, data_t[j]['referrer']).groupdict()['path'] 
            except:
                ref = data_t[j]['referrer']
            if ref==data_t[i]['path']:
                mat[i,j] = 1
                continue

            if data_t[j]['path']==data_t[i]['path'] and data_t[j]['referrer']==data_t[i]['referrer']:
                if data_t[j]['type']==3:
                    mat[i,j] = 2
                    continue
                if data_t[j]['type']==data_t[i]['type'] and data_t[i]['type']==1:
                    d1 = copy.deepcopy(data_t[i])
                    d2 = copy.deepcopy(data_t[j])
                    #d1 = data_t[i].values()
                    #d2 = data_t[j].values()
                    if d1.pop('timestamp')!=d2.pop('timestamp'):
                        d1.pop('userId')
                        d2.pop('userId')
                        try:
                            d1.pop('originalTimestamp')
                            d2.pop('originalTimestamp')
                        except:
                            pass
                        if d1==d2:
                            mat[i,j]=3
                            continue
                if data_t[j]['type']==2:
                    if data_t[j]['event']==2:
                        mat[i,j]=4
                        continue
                    if data_t[j]['event']==3:
                        mat[i,j]=5
                        continue
                    if data_t[j]['event']==1:
                        mat[i,j]=6
                        continue
    return mat

In [313]:
def get_paths(mat_t):
    N = mat_t.shape[0]
    b = np.ones(N)
    paths = []
    for j in range(N-1,0,-1):
        i=N-1
        path = []
        #print
        while (i>0) and (j>=1):
            #print j
            for i in range(j-1,-1,-1):
                #print "\t %d" % i, mat_t[i,j]
                if mat_t[i,j]==0:
                    continue
                if mat_t[i,j] == 1:
                    path += [data[j]['path']]
                    break
                if mat_t[i,j] == 2:
                    path += ['identify']
                    break
                if mat_t[i,j] == 3:
                    #path += [data[j]['path']]
                    break
                    pass
                if mat_t[i,j] == 4:
                    path += [u'Requested Treatment']
                    break
                if mat_t[i,j] == 5:
                    path += [u'Requested Callback']
                    break
                if mat_t[i,j] == 6:
                    #print data[j]
                    path += [data[j]['properties']['button_type']]
                    break                  
            if sum(mat_t[0:i,j])==0:
                path += [data[j]['path']]
                path += [data[j]['referrer']]
                break
            if i==0:
                if data[0]['type']==1:
                    path+=[data[0]['path']]
                    try:
                        path+=[re.match(r, data[0]['referrer']).groupdict()['path'] ]
                    except:
                        path+=[data[0]['referrer']]
                if data[0]['type']==3:
                    path+=['identify']
            j=i
            #print "\t \t" ,i,j, b
            #print (i!=0 or j!=1), '=', (i!=0), (j!=1)
        paths += [path]
    return paths

In [4]:
def filter_paths(paths_t):
    inds=[]
    paths=sorted(paths_t, key=lambda x: len(x),reverse=True)
    for i in range(len(paths)-1,-1,-1):
        for j in range(i-1,-1,-1):
            l = len(paths[j]) - len(paths[i])
            if paths[j][l:]==paths[i]:
                inds+=[i]
                break
    for i in inds:
        tenp=paths.pop(i)
    return paths

In [114]:
def simplify_path(paths_t):
    for path in paths_t:
        for i in range(0, len(path)):
            try:
                ref = re.match(r, path[i]).groupdict()['ref']
                if ref == 'bookimed':
                    path[i] = re.match(r, path[i]).groupdict()['tail']
                else:
                    path[i] = ref
            except:
                pass
    return paths_t

In [5]:
def paths_to_str(paths_t):
    str_paths = []
    for i in paths_t:
        str_paths += [' , '.join(list(reversed(i)))]
    return str_paths

In [6]:
files = [os.path.basename(x) for x in glob.glob('./reg_users/*.json')]

In [314]:
all_paths = []
output = open('paths.txt', 'w')
for file_name in files[:2000]:
    f = open('./un_users/%s'%file_name)
    context = f.readlines()
    data = []
    for i in context[:]:
        try:
            data += [json.loads(i.replace(',\n','').replace('\n',''))]
        except:
            pass
    data=sorted(data, key=lambda x: x['timestamp'])
    for path in paths_to_str(simplify_path(filter_paths(get_paths(get_relation_mat(data))))):
        output.write(path.encode('utf-8') + '\n')
        all_paths += [path]
    output.write('%d \n------------\n'%files.index(file_name))
output.close()

In [195]:
path.encode('utf-8')

'google , google , auto_help'

In [315]:
len(all_paths)

6654

In [316]:
len(set(all_paths))

5707

In [198]:
srt = sorted(list(set(all_paths)))

In [ ]:
for i in srt[-100:]:
    print i
    print

In [321]:
f = open('./un_users/%s'%files[745])
context = f.readlines()
data = []
for i in context[:]:
    try:
        data += [json.loads(i.replace(',\n','').replace('\n',''))]
    except:
        pass
data=sorted(data, key=lambda x: x['timestamp'])

In [322]:
get_paths(get_relation_mat(data))

[[u'/clientLK/profile/', u''],
 [u'/clientLK/profile/',
  u'/clientLK/requests/',
  u'/clientLK/tprograms/',
  u'/clientLK/chat/',
  u'/clientLK/chat/',
  u'http://ru.bookimed.com/clientLK/profile/'],
 [u'/clientLK/requests/',
  u'/clientLK/tprograms/',
  u'/clientLK/chat/',
  u'/clientLK/chat/',
  u'http://ru.bookimed.com/clientLK/profile/'],
 [u'/clientLK/tprograms/',
  u'/clientLK/chat/',
  u'/clientLK/chat/',
  u'http://ru.bookimed.com/clientLK/profile/'],
 [u'/clientLK/chat/',
  u'/clientLK/chat/',
  u'http://ru.bookimed.com/clientLK/profile/'],
 [u'/clientLK/profile/',
  u'/clientLK/profile/',
  u'http://ru.bookimed.com/clientLK/requests/'],
 [u'/clientLK/requests/',
  u'/clientLK/requests/',
  u'http://ru.bookimed.com/clientLK/tprograms/'],
 [u'/clientLK/tprograms/',
  u'/clientLK/tprograms/',
  u'http://ru.bookimed.com/clientLK/chat/'],
 [u'/clientLK/chat/',
  u'/clientLK/chat/',
  u'http://ru.bookimed.com/clientLK/last_step/'],
 [u'/clientLK/last_step/',
  u'/clientLK/last_ste

In [319]:
paths=filter_paths(get_paths(get_relation_mat(data)))

In [320]:
paths_to_str(simplify_path(filter_paths(get_paths(get_relation_mat(data)))))

[u'/ , /clinic/medicinskij-centr-samsung/ , /clinic/medicinskij-centr-samsung/ , clinic_get_cost_header',
 u'/clientLK/docs/ , /',
 u' , /clinics/country=republic-of-korea/illness=epilepsiya/']

In [307]:
m = get_relation_mat(data)
print m

[[ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  1.]
 [ 0.  0.  0.  0.  6.]
 [ 0.  0.  0.  0.  0.]]


In [278]:
sum(m[0:2,3])

0.0

In [306]:
paths

[[u'clinic_get_cost_header',
  u'/clinic/medicinskij-centr-samsung/',
  u'/clinic/medicinskij-centr-samsung/',
  u'http://ru.bookimed.com/'],
 [u'/clinic/medicinskij-centr-samsung/',
  u'',
  u'/',
  u'http://ru.bookimed.com/clientLK/docs/?utm_term=from_popup_go_to_docs'],
 [u'/clinic/medicinskij-centr-samsung/',
  u'',
  u'/clinics/country=republic-of-korea/illness=epilepsiya/',
  u'']]

In [323]:
cut = [[i['timestamp'][:-8], i['referrer'], i['path'], i['type']] for i in data]

In [324]:
for c in cut:
    print cut.index(c), c
    print

0 [u'2015-11-17T08:30:00.', u'http://yandex.ru/clck/', u'/clinic/onkologicheskij-centr-andersena/', 1]

1 [u'2015-11-17T08:30:53.', u'http://ru.bookimed.com/clinic/onkologicheskij-centr-andersena/', u'/countries/', 1]

2 [u'2015-11-17T08:30:58.', u'http://ru.bookimed.com/countries/', u'/clinics/country=united-states/', 1]

3 [u'2015-11-17T08:31:50.', u'http://ru.bookimed.com/clinics/country=united-states/', u'/clinic/gospital-dzhona-hopkinsa/', 1]

4 [u'2015-11-17T08:32:57.', u'http://ru.bookimed.com/clinics/country=united-states/', u'/clinic/gospital-dzhona-hopkinsa/', 2]

5 [u'2015-11-17T08:32:59.', u'http://ru.bookimed.com/countries/', u'/clinics/country=united-states/', 1]

6 [u'2015-11-17T08:33:12.', u'http://ru.bookimed.com/clinics/country=united-states/', u'/clinic/onkologicheskij-centr-andersena/', 1]

7 [u'2015-11-17T08:33:26.', u'http://ru.bookimed.com/clinic/onkologicheskij-centr-andersena/', u'/review/620/', 1]

8 [u'2015-11-17T08:33:42.', u'http://ru.bookimed.com/clinics/c

In [327]:
data[12]['referrer']

u'http://ru.bookimed.com/clientLK/?action=login&login=ok.arhsc@mail.ru&hash=698773ad9ef9301b5bc0818ca9060e43'

In [359]:
bookimed_pattern = u'^http[s]*://[ru.]*bookimed.com(?P<tail>[\w/=?..&@-]*)'

In [360]:
b_r = re.compile(bookimed_pattern)
re.match(b_r, data[12]['referrer']).groupdict()['tail']

u'/clientLK/?action=login&login=ok.arhsc@mail.ru&hash=698773ad9ef9301b5bc0818ca9060e43'

In [252]:
data[0]

{u'fingerPrint': u'3ac84de680888c96e2d831413f8451ff',
 u'ip': u'95.135.169.42',
 u'path': u'/clinic/medicinskij-centr-samsung/',
 u'referrer': u'',
 u'search': u'',
 u'timestamp': u'2016-05-19T19:26:23.492+0000',
 u'type': 1,
 u'url': u'http://ru.bookimed.com/clinic/medicinskij-centr-samsung/',
 u'userId': None}